In [1]:
import os

import numpy as np
import torch

from tqdm import tqdm

In [2]:
info = torch.load("./data/dataset.pt")
info


,pdb,mutation,affinity_wt,affinity_mut,temp,ddg,daffinity
0,1CSE_E_I,LI38G,1.120000e-12,5.260000e-11,294.0,2.248833,5.148000e-11
1,1CSE_E_I,LI38S,1.120000e-12,8.330000e-12,294.0,1.172229,7.210000e-12
2,1CSE_E_I,LI38P,1.120000e-12,1.020000e-07,294.0,6.671276,1.019989e-07
3,1CSE_E_I,LI38I,1.120000e-12,1.720000e-10,294.0,2.940988,1.708800e-10
4,1CSE_E_I,LI38D,1.120000e-12,1.920000e-09,294.0,4.350434,1.918880e-09
...,...,...,...,...,...,...,...
5830,3QIB_ABP_CD,KP8R,5.500000e-06,2.400000e-04,298.0,2.235909,2.345000e-04
5831,3QIB_ABP_CD,TP11A,5.500000e-06,1.100000e-03,298.0,3.137419,1.094500e-03
5832,3QIB_ABP_CD,TP11S,5.500000e-06,3.380000e-05,298.0,1.075181,2.830000e-05
5833,3QIB_ABP_CD,TP11N,5.500000e-06,4.340000e-05,298.0,1.223219,3.790000e-05


In [3]:

entry = info.iloc[0]
type(entry['ddg'])
entry['pdb'].split('_')

['1CSE', 'E', 'I']

In [4]:
from data import *

train_dataset = SiameseProteinGraphDataset(info, feature_path="./data/", radius=15)

index = 3
train_dataset[index]

(Data(edge_index=[2, 21299], name='1CSE', seq='AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVGGASFVAGEAYNTDGNGHGTHVAGTVAALDNTTGVLGVAPSVSLYAVKVLNSSGSGSYSGIVSGIEWATTNGMDVINMSLGGASGSTAMKQAVDNAYARGVVVVAAAGNSGNSGSTNTIGYPAKYDSVIAVGAVDSNSNRASFSSVGAELEVMAPGAGVYSTYPTNTYATLNGTSMASPHVAGAAALILSKHPNLSASQVRNRLSSTATYLGSSFYYGKGLINVEAAAQKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVRVFYNPGTNVVNHVPHVG', coord=[337, 5, 3], node_feat=[337, 1033]),
 Data(edge_index=[2, 21299], name='1CSE_E_I_LI38I', seq='AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVGGASFVAGEAYNTDGNGHGTHVAGTVAALDNTTGVLGVAPSVSLYAVKVLNSSGSGSYSGIVSGIEWATTNGMDVINMSLGGASGSTAMKQAVDNAYARGVVVVAAAGNSGNSGSTNTIGYPAKYDSVIAVGAVDSNSNRASFSSVGAELEVMAPGAGVYSTYPTNTYATLNGTSMASPHVAGAAALILSKHPNLSASQVRNRLSSTATYLGSSFYYGKGLINVEAAAQKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTIDLRYNRVRVFYNPGTNVVNHVPHVG', coord=[337, 5, 3], node_feat=[337, 1033]),
 tensor(2.9410))

In [5]:
from torch_geometric.loader import DataLoader

batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

print(f"train_dataset: {train_dataset.__len__()}")  # total_data * (fold-1)/fold
print(f"train_dataloader: {train_dataloader.__len__()}")  # num_samples / batch_size


train_dataset: 5835
train_dataloader: 1459


In [6]:
from model import *

model = SiameseGPSite()


In [7]:
learning_rate = 1e-3
beta12 = (0.9, 0.99)
num_epochs = 25
epochs = 25

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=beta12, weight_decay=1e-5, eps=1e-5)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(train_dataloader), epochs=epochs)  # ???



In [8]:
# test
train_data = next(iter(train_dataloader))
model.eval()
with torch.no_grad():
    # train_data = train_data.to(device)
    wt_graph, mut_graph, y = train_data
    print(wt_graph)
    print(mut_graph)

    pred = model(wt_graph, mut_graph)

    print(f"Predicted: {pred}\nActual: {y}")
    # print((pred == y))
    # print(((pred == y)).sum())
    


DataBatch(edge_index=[2, 85196], name=[4], seq=[4], coord=[1348, 5, 3], node_feat=[1348, 1033], batch=[1348], ptr=[5])
DataBatch(edge_index=[2, 85196], name=[4], seq=[4], coord=[1348, 5, 3], node_feat=[1348, 1033], batch=[1348], ptr=[5])
Predicted: tensor([-0.0533, -0.0532, -0.0533, -0.0531])
Actual: tensor([2.2488, 1.1722, 6.6713, 2.9410])
